In [2]:
#ex1
# 
from pyspark.sql import SparkSession

# Création de la SparkSession
spark = SparkSession.builder \
    .appName("Lecture WorldCitiesPop") \
    .getOrCreate()

# Lire le fichier worldcitiespop.txt
cities_rdd = spark.sparkContext.textFile("C:\EXERCICES\pyspark\PySpark\worldcitiespop.txt")

# Afficher la première ligne avec first()
print("Première ligne du fichier worldcitiespop.txt :")
print(cities_rdd.first())

# Arrêter la session Spark
spark.stop()

Première ligne du fichier worldcitiespop.txt :
Country,City,AccentCity,Region,Population,Latitude,Longitude


In [ ]:
#ex1 première méthode
# 
from pyspark.sql import SparkSession
from pyspark import SparkContext

# Création de la SparkSession
spark = SparkSession.builder \
    .appName("Nettoyage WorldCitiesPop") \
    .getOrCreate()

# Définir le chemin du fichier
file_path = "C:/EXERCICES/pyspark/PySpark/worldcitiespop.txt"

# Lire le fichier
cities_rdd = spark.sparkContext.textFile(file_path)

# Fonction pour parser une ligne et vérifier si la population est valide
def parse_line(line):
    try:
        fields = line.split(',')
        if len(fields) >= 5:  # Vérifier qu'il y a au moins 5 champs
            population = fields[4]
            # Vérifier si la population est un nombre non vide et positif
            if population and float(population) > 0:
                return True
    except (ValueError, IndexError):
        pass
    return False

# Filtrer les lignes avec population valide
# Ignorer la première ligne (en-têtes)
header = cities_rdd.first()
valid_cities = cities_rdd.filter(lambda x: x != header).filter(parse_line)

# Afficher quelques statistiques
total_lines = cities_rdd.count() - 1  # -1 pour l'en-tête
valid_lines = valid_cities.count()

print("Statistiques du nettoyage :")
print("-" * 30)
print(f"Nombre total de lignes (sans en-tête): {total_lines}")
print(f"Nombre de lignes avec population valide: {valid_lines}")
print(f"Pourcentage de lignes valides: {(valid_lines/total_lines)*100:.2f}%")

# Afficher quelques exemples de lignes valides
print("\nExemples de lignes valides:")
print("-" * 30)
for line in valid_cities.take(5):
    print(line)

# Sauvegarder les résultats si nécessaire
# valid_cities.saveAsTextFile("C:/EXERCICES/pyspark/PySpark/valid_cities")

# Arrêter la session Spark
spark.stop()

Statistiques du nettoyage :
------------------------------
Nombre total de lignes (sans en-tête): 3173958
Nombre de lignes avec population valide: 47980
Pourcentage de lignes valides: 1.51%

Exemples de lignes valides:
------------------------------
ad,andorra la vella,Andorra la Vella,07,20430,42.5,1.5166667
ad,canillo,Canillo,02,3292,42.5666667,1.6
ad,encamp,Encamp,03,11224,42.5333333,1.5833333
ad,la massana,La Massana,04,7211,42.55,1.5166667
ad,les escaldes,Les Escaldes,08,15854,42.5,1.5333333


In [5]:
#ex2 avec math

from pyspark import *
from math import *

# Création du SparkContext
sc = SparkContext("local", "Exercice 2")

# Définir le chemin du fichier
file_path = "C:/EXERCICES/pyspark/PySpark/worldcitiespop.txt"

# Lire le fichier
cities_rdd = sc.textFile(file_path)

# Fonction pour parser une ligne et vérifier si la population est valide
def parse_line(line):
    try:
        fields = line.split(',')
        if len(fields) >= 5:  # Vérifier qu'il y a au moins 5 champs
            population = fields[4]
            # Vérifier si la population est un nombre non vide et positif
            if population and float(population) > 0:
                return True
    except (ValueError, IndexError):
        pass
    return False

# Filtrer les lignes avec population valide
# Ignorer la première ligne (en-têtes)
header = cities_rdd.first()
valid_cities = cities_rdd.filter(lambda x: x != header).filter(parse_line)

# Afficher quelques statistiques
total_lines = cities_rdd.count() - 1  # -1 pour l'en-tête
valid_lines = valid_cities.count()

print("Statistiques du nettoyage :")
print("-" * 30)
print(f"Nombre total de lignes (sans en-tête): {total_lines}")
print(f"Nombre de lignes avec population valide: {valid_lines}")
print(f"Pourcentage de lignes valides: {(valid_lines/total_lines)*100:.2f}%")

# Afficher quelques exemples de lignes valides
print("\nExemples de lignes valides:")
print("-" * 30)
for line in valid_cities.take(5):
    print(line)

# Arrêter le SparkContext
sc.stop()

Statistiques du nettoyage :
------------------------------
Nombre total de lignes (sans en-tête): 3173958
Nombre de lignes avec population valide: 47980
Pourcentage de lignes valides: 1.51%

Exemples de lignes valides:
------------------------------
ad,andorra la vella,Andorra la Vella,07,20430,42.5,1.5166667
ad,canillo,Canillo,02,3292,42.5666667,1.6
ad,encamp,Encamp,03,11224,42.5333333,1.5833333
ad,la massana,La Massana,04,7211,42.55,1.5166667
ad,les escaldes,Les Escaldes,08,15854,42.5,1.5333333


In [6]:
#ex 3 et ex 4

from pyspark import *
from math import *

# Création du SparkContext
sc = SparkContext("local", "Exercices 3 et 4")

# Lire le fichier
cities_rdd = sc.textFile("C:/EXERCICES/pyspark/PySpark/worldcitiespop.txt")

# Fonction pour extraire la population si valide
def get_population(line):
    try:
        fields = line.split(',')
        if len(fields) >= 5:
            population = fields[4]
            if population and float(population) > 0:
                return float(population)
    except (ValueError, IndexError):
        pass
    return None

# Ignorer l'en-tête et extraire les populations valides
header = cities_rdd.first()
populations = cities_rdd.filter(lambda x: x != header) \
                      .map(get_population) \
                      .filter(lambda x: x is not None)

### Exercice 3 : Statistiques ###
# Calculer les statistiques
count = populations.count()
sum_pop = populations.sum()
min_pop = populations.min()
max_pop = populations.max()
avg_pop = sum_pop / count

print("Statistiques sur les populations:")
print(f"count: {count}")
print(f"mean: {avg_pop}")
print(f"min: {min_pop}")
print(f"max: {max_pop}")

### Exercice 4 : Histogrammes ###
# Fonction pour déterminer la classe (en échelle logarithmique)
def get_class(population):
    if population > 0:
        return floor(log10(population))
    return 0

# Calculer l'histogramme
histogram = populations.map(lambda x: (get_class(x), 1)) \
                      .reduceByKey(lambda a, b: a + b) \
                      .sortByKey() \
                      .collect()

print("\nHistogramme des populations (échelle logarithmique):")
print("Classe | Nombre de villes")
print("-" * 30)
for classe, count in histogram:
    print(f"10^{classe} | {count}")

# Arrêter le SparkContext
sc.stop()

Statistiques sur les populations:
count: 47980
mean: 47719.57063359733
min: 7.0
max: 31480498.0

Histogramme des populations (échelle logarithmique):
Classe | Nombre de villes
------------------------------
10^0 | 5
10^1 | 174
10^2 | 2187
10^3 | 20537
10^4 | 21550
10^5 | 3248
10^6 | 269
10^7 | 10


In [7]:
#ex5

from pyspark import *
from math import *

# Création du SparkContext
sc = SparkContext("local", "Exercice 5")

# Lire le fichier
cities_rdd = sc.textFile("C:/EXERCICES/pyspark/PySpark/worldcitiespop.txt")

# Fonction pour extraire les informations de ville avec population
def parse_city(line):
    try:
        fields = line.split(',')
        if len(fields) >= 6:  # Au moins 6 champs nécessaires
            country = fields[0]
            city = fields[1]
            accent_city = fields[2]
            region = fields[3]
            population = fields[4]
            if population and float(population) > 0:
                return (country, city, accent_city, region, float(population))
    except (ValueError, IndexError):
        pass
    return None

# Ignorer l'en-tête et extraire les villes avec population valide
header = cities_rdd.first()
valid_cities = cities_rdd.filter(lambda x: x != header) \
                        .map(parse_city) \
                        .filter(lambda x: x is not None)

# Trier par population décroissante et prendre les 10 premières villes
top_10_cities = valid_cities.sortBy(lambda x: -x[4]) \
                           .take(10)

# Afficher les résultats
print("Les 10 villes les plus peuplées:")
print("-" * 50)
print("Pays | Ville | Ville (avec accents) | Région | Population")
print("-" * 50)
for city in top_10_cities:
    print(f"{city[0]}, {city[1]}, {city[2]}, {city[3]}, {int(city[4])}")

# Arrêter le SparkContext
sc.stop()


Les 10 villes les plus peuplées:
--------------------------------------------------
Pays | Ville | Ville (avec accents) | Région | Population
--------------------------------------------------
jp, tokyo, Tokyo, 40, 31480498
cn, shanghai, Shanghai, 23, 14608512
in, bombay, Bombay, 16, 12692717
pk, karachi, Karachi, 05, 11627378
in, delhi, Delhi, 07, 10928270
in, new delhi, New Delhi, 07, 10928270
ph, manila, Manila, D9, 10443877
ru, moscow, Moscow, 48, 10381288
kr, seoul, Seoul, 11, 10323448
br, sao paulo, S�o Paulo, 27, 10021437


In [8]:
#ex 6

from pyspark import *
from math import *

# Création du SparkContext
sc = SparkContext("local", "Exercice 6 - Re-cleaning")

# Lire le fichier
cities_rdd = sc.textFile("C:/EXERCICES/pyspark/PySpark/worldcitiespop.txt")

# Fonction pour parser les villes avec leurs coordonnées
def parse_city_with_coords(line):
    try:
        fields = line.split(',')
        if len(fields) >= 7 and fields[4]:  # Vérifier population et coordonnées
            country = fields[0]
            city = fields[1]
            accent_city = fields[2]
            region = fields[3]
            population = float(fields[4])
            latitude = float(fields[5])
            longitude = float(fields[6])
            if population > 0:
                # Clé: coordonnées arrondies pour regrouper les villes proches
                key = (round(latitude, 1), round(longitude, 1))
                # Valeur: toutes les informations de la ville
                value = (country, city, accent_city, region, population, latitude, longitude)
                return (key, value)
    except (ValueError, IndexError):
        pass
    return None

# Traitement des données
header = cities_rdd.first()
cleaned_cities = cities_rdd.filter(lambda x: x != header) \
    .map(parse_city_with_coords) \
    .filter(lambda x: x is not None) \
    .groupByKey() \
    .mapValues(lambda cities: max(cities, key=lambda x: x[4])) \
    .values()

# Calculer les statistiques
populations = cleaned_cities.map(lambda x: x[4])
count = populations.count()
mean = populations.mean()
stdev = populations.stdev()
max_pop = populations.max()
min_pop = populations.min()

print(f"(count: {count}, mean: {mean:.2f}, stdev: {stdev:.2f}, max: {max_pop}, min: {min_pop})")

# Calculer l'histogramme
def get_class(population):
    return floor(log10(population))

histogram = populations.map(lambda x: (get_class(x), 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .sortByKey() \
    .collect()

print(histogram)

# Afficher les 20 plus grandes villes
top_cities = cleaned_cities.sortBy(lambda x: -x[4]).take(20)

# Formater et afficher les résultats
for city in top_cities:
    country, city_name, accent_city, region, pop, lat, lon = city
    print(f"{country},{city_name},{accent_city},{region},{int(pop)},{lat},{lon}")

# Arrêter le SparkContext
sc.stop()

(count: 37575, mean: 57219.79, stdev: 340982.15, max: 31480498.0, min: 7.0)
[(0, 4), (1, 61), (2, 1346), (3, 14449), (4, 18362), (5, 3078), (6, 265), (7, 10)]
jp,tokyo,Tokyo,40,31480498,35.685,139.751389
cn,shanghai,Shanghai,23,14608512,31.045556,121.399722
in,bombay,Bombay,16,12692717,18.975,72.825833
pk,karachi,Karachi,05,11627378,24.9056,67.0822
in,delhi,Delhi,07,10928270,28.666667,77.216667
in,new delhi,New Delhi,07,10928270,28.6,77.2
ph,manila,Manila,D9,10443877,14.6042,120.9822
ru,moscow,Moscow,48,10381288,55.752222,37.615556
kr,seoul,Seoul,11,10323448,37.5985,126.9783
br,sao paulo,S�o Paulo,27,10021437,-23.473293,-46.665803
tr,istanbul,Istanbul,34,9797536,41.018611,28.964722
ng,lagos,Lagos,05,8789133,6.453056,3.395833
mx,mexico,Mexico,09,8720916,19.434167,-99.138611
id,jakarta,Jakarta,04,8540306,-6.174444,106.829444
us,new york,New York,NY,8107916,40.7141667,-74.0063889
cd,kinshasa,Kinshasa,06,7787832,-4.3,15.3
eg,cairo,Cairo,11,7734602,30.05,31.25
pe,lima,Lima,15,7646786,-12.05